# Get_Original_AlphaNet_Data

In [1]:
Alpha_Name = "5_Input_20220110"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "1d_ret"

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

# Get_Original_AlphaNet_Data

In [3]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.makedirs(path+i)

In [4]:
df = pd.read_csv(path+"data.csv")
df.dropna(inplace=True,axis=0,how="any")
df

,timestamp,ticker,high,low,close,volume,pct_chg,1d_ret
0,2015-01-06 09:30:00,1,0.016226,-0.026215,1138.28,2860436.43,1.1364,-0.018297
1,2015-01-06 09:30:00,2,0.025490,-0.046275,1838.34,6560835.70,7.2662,-0.023288
2,2015-01-06 09:30:00,4,0.013488,-0.024780,63.76,28095.12,0.7707,0.051118
3,2015-01-06 09:30:00,5,0.000000,0.000000,38.00,0.00,0.0000,0.000000
4,2015-01-06 09:30:00,6,0.043778,-0.008444,225.00,1078647.12,0.4255,-0.028571
...,...,...,...,...,...,...,...,...
5357699,2021-07-01 09:30:00,688700,0.020982,-0.118958,53.38,64336.83,14.5494,-0.040395
5357700,2021-07-01 09:30:00,688777,0.006990,-0.054014,94.42,19001.61,4.6111,-0.005927
5357701,2021-07-01 09:30:00,688788,0.021406,-0.062303,146.22,13572.56,5.1954,-0.025238
5357702,2021-07-01 09:30:00,688819,0.010853,-0.026578,45.15,47249.01,1.8544,-0.022042


In [5]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(one_data):
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [6]:
if __name__=="__main__":
    pool = mp.Pool(10)
    result = {}
    t1 = time.time()
    grouped = df.groupby("ticker")
    for ticker , one_data in tqdm(grouped):
        result[ticker] = (pool.apply_async(test, args=(one_data,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 98102.14it/s]

并行执行时间： 40


,,high,low,close,volume,pct_chg,high,low,close,volume,pct_chg,...,low,close,volume,pct_chg,high,low,close,volume,pct_chg,1d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,0.016226,-0.026215,1138.28,2860436.43,1.1364,0.038654,-0.014573,1121.23,2166421.40,-1.4981,...,-0.017313,984.80,608715.68,0.9468,0.016485,-0.007889,991.20,887743.10,0.6494,0.000717
2015-02-17 09:30:00,1,0.038654,-0.014573,1121.23,2166421.40,-1.4981,0.022611,-0.011628,1099.91,1700120.67,-1.9011,...,-0.007889,991.20,887743.10,0.6494,0.006461,-0.013649,989.07,721202.39,-0.2151,0.005731
2015-02-25 09:30:00,1,0.022611,-0.011628,1099.91,1700120.67,-1.9011,0.040782,-0.004008,1062.96,1407714.21,-3.3592,...,-0.013649,989.07,721202.39,-0.2151,0.008581,-0.004286,994.04,680491.59,0.5029,-0.017094
2015-02-26 09:30:00,1,0.040782,-0.004008,1062.96,1407714.21,-3.3592,0.052385,-0.024536,1071.49,2508500.23,0.8021,...,-0.004286,994.04,680491.59,0.5029,0.018112,-0.002896,980.54,577392.33,-1.3581,0.018841
2015-02-27 09:30:00,1,0.052385,-0.024536,1071.49,2508500.23,0.8021,0.018962,-0.018276,1049.46,1553290.86,-2.0557,...,-0.002896,980.54,577392.33,-1.3581,0.000710,-0.033409,999.73,1025544.24,1.9565,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,688981,0.002329,-0.023647,55.82,209785.85,0.6310,0.020050,-0.004654,55.86,260448.82,0.0717,...,-0.024428,58.54,511088.21,2.5578,0.024437,-0.002600,57.70,273520.23,-1.4349,0.036184
2021-06-28 09:30:00,688981,0.020050,-0.004654,55.86,260448.82,0.0717,0.023965,-0.000545,55.08,203770.43,-1.3963,...,-0.002600,57.70,273520.23,-1.4349,0.011465,-0.026134,59.31,474988.89,2.7903,-0.012531
2021-06-29 09:30:00,688981,0.023965,-0.000545,55.08,203770.43,-1.3963,0.003637,-0.009093,54.99,121147.00,-0.1634,...,-0.026134,59.31,474988.89,2.7903,0.019133,-0.003386,59.06,341515.10,-0.4215,0.009983


In [7]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.dropna(axis=0,inplace=True) # delete none
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,141,142,143,144,145,146,147,148,149,target
0,2015-02-16 09:30:00,1,0.016226,-0.026215,1138.28,2860436.43,1.1364,0.038654,-0.014573,1121.23,...,-0.017313,984.80,608715.68,0.9468,0.016485,-0.007889,991.20,887743.10,0.6494,0.000717
1,2015-02-17 09:30:00,1,0.038654,-0.014573,1121.23,2166421.40,-1.4981,0.022611,-0.011628,1099.91,...,-0.007889,991.20,887743.10,0.6494,0.006461,-0.013649,989.07,721202.39,-0.2151,0.005731
2,2015-02-25 09:30:00,1,0.022611,-0.011628,1099.91,1700120.67,-1.9011,0.040782,-0.004008,1062.96,...,-0.013649,989.07,721202.39,-0.2151,0.008581,-0.004286,994.04,680491.59,0.5029,-0.017094
3,2015-02-26 09:30:00,1,0.040782,-0.004008,1062.96,1407714.21,-3.3592,0.052385,-0.024536,1071.49,...,-0.004286,994.04,680491.59,0.5029,0.018112,-0.002896,980.54,577392.33,-1.3581,0.018841
4,2015-02-27 09:30:00,1,0.052385,-0.024536,1071.49,2508500.23,0.8021,0.018962,-0.018276,1049.46,...,-0.002896,980.54,577392.33,-1.3581,0.000710,-0.033409,999.73,1025544.24,1.9565,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230320,2021-06-25 09:30:00,688981,0.002329,-0.023647,55.82,209785.85,0.6310,0.020050,-0.004654,55.86,...,-0.024428,58.54,511088.21,2.5578,0.024437,-0.002600,57.70,273520.23,-1.4349,0.036184
5230321,2021-06-28 09:30:00,688981,0.020050,-0.004654,55.86,260448.82,0.0717,0.023965,-0.000545,55.08,...,-0.002600,57.70,273520.23,-1.4349,0.011465,-0.026134,59.31,474988.89,2.7903,-0.012531
5230322,2021-06-29 09:30:00,688981,0.023965,-0.000545,55.08,203770.43,-1.3963,0.003637,-0.009093,54.99,...,-0.026134,59.31,474988.89,2.7903,0.019133,-0.003386,59.06,341515.10,-0.4215,0.009983
5230323,2021-06-30 09:30:00,688981,0.003637,-0.009093,54.99,121147.00,-0.1634,0.007483,-0.003833,54.79,...,-0.003386,59.06,341515.10,-0.4215,0.014008,-0.009283,59.25,291408.81,0.3217,0.026973


In [8]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(10)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 4555.79it/s]


Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2017-01-01_2017-03-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2016-03-01_2016-06-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2017-03-01_2017-06-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2018-01-01_2018-03-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2015-09-01_2016-01-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2016-09-01_2017-01-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2017-06-01_2017-09-01.par
Save to  /home/wuwenjun/Data/5_Input_20220110/Data/2017-09-01_2018-01-01.par

# Get Factor

In [9]:
day = 30
stride = 10
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(stride,data_length = 30,kernal_size = None):
    """
    data_length: days
    kernal_size: how many days'moving
    stride
    """
    step_list = []
    start , end = 0 , 0
    if kernal_size == None:
        kernal_size = stride
    while end < data_length:
        end = start + kernal_size
        if end >= data_length:
            end = data_length
            start = end - kernal_size
        step_list.append([start,end])
        start += stride
    return step_list

In [10]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 107018.55it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [25]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list,output_path):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov"))
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(stride = stride,data_length=data_length)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = corr(x,num,num_rev,step_list,output_path+name)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [13]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 3/3 [00:00<00:00, 13.68it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Cov/2015-01-01_2015-03-01.par


100%|██████████| 3/3 [00:02<00:00,  1.21it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Cov/2016-01-01_2016-03-01.par


 67%|██████▋   | 2/3 [00:02<00:01,  1.22s/it]

...

save to  /home/wuwenjun/Data/5_Input_20220110/Decay/2020-03-01_2020-06-01.par
save to  /home/wuwenjun/Data/5_Input_20220110/Decay/2019-09-01_2020-01-01.par


 33%|███▎      | 1/3 [00:00<00:00,  7.62it/s]

save to  /home/wuwenjun/Data/5_Input_20220110/Decay/2020-06-01_2020-09-01.par


100%|██████████| 3/3 [00:00<00:00,  8.70it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Decay/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/5_Input_20220110/Decay/2020-09-01_2021-01-01.par
并行执行时间： 37


In [26]:
t1 = time.time()
pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
#     test(data_path,alpha,output,"Corr")
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 3/3 [00:00<00:00, 22.75it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Corr/2015-01-01_2015-03-01.par


100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Corr/2015-03-01_2015-06-01.par


100%|██████████| 3/3 [00:03<00:00,  1.14s/it]


...

save to  /home/wuwenjun/Data/5_Input_20220110/Corr/2020-09-01_2021-01-01.par


100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


save to  /home/wuwenjun/Data/5_Input_20220110/Corr/2021-01-01_2021-03-01.par


100%|██████████| 3/3 [00:05<00:00,  1.79s/it]


save to  /home/wuwenjun/Data/5_Input_20220110/Corr/2021-03-01_2021-06-01.par
并行执行时间： 144


# Feat Concat ( Save to Feat_Concat file)

In [14]:
os.mkdir(path+"Feat_Concat")

In [4]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 49188.95it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [29]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Corr','Std','Zscore','Return','Decay']

final = []
def test(alpha,dir_list):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
    print(path+"Feat_Concat/"+alpha)
    return None
for alpha in tqdm(path_list):
#     test(alpha,dir_list)
    pool.apply_async(test, args=(alpha,dir_list))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 28465.65it/s]


/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2015-01-01_2015-03-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2016-01-01_2016-03-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2017-01-01_2017-03-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2017-03-01_2017-06-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2015-03-01_2015-06-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2016-09-01_2017-01-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2016-06-01_2016-09-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2018-01-01_2018-03-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2016-03-01_2016-06-01.par/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2015-09-01_2016-01-01.par

/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2017-06-01_2017-09-01.par
/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2017-09-01_2018-01-01.par
/home/wuwenjun/Data/5_Inp

In [46]:
a = pd.read_parquet("/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/2020-09-01_2021-01-01.par").set_index(["timestamp","ticker"])

In [55]:
for i in tqdm(path_list):
    a = pd.read_parquet("/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/" + i).set_index(["timestamp","ticker"])
    index = a.index
    a = np.array(a).reshape(np.array(a).shape[0],40,3)
    a = a.transpose(0,2,1)
    a = a.reshape(a.shape[0],-1)
    a = pd.DataFrame(a)
    a.index = index
    a.columns = [str(i) for i in a.columns]
    a.to_parquet("/home/wuwenjun/Data/5_Input_20220110/Feat_Concat/" + i)

100%|██████████| 26/26 [01:17<00:00,  2.98s/it]


In [ ]:
pool = mp.Pool(10)
result = {}
t1 = time.time()
def test(j):
    return pd.read_parquet(j)
for i in tqdm(path_list):
    result[i] = (pool.apply_async(test, args=(path+"Feat_Concat/"+i,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))
df_list = []
for i in tqdm(result):
    df_list.append(result[i].get())
final_df = pd.concat(df_list,axis=0)
final_df.reset_index(inplace=True)
final_df["timestamp"] = pd.to_datetime(final_df["timestamp"])
final_df["ticker"] = final_df["ticker"].apply(lambda x: str(x).zfill(6))
final_df.set_index(["timestamp","ticker"],inplace=True)
final_df

In [7]:
from AlphaNet.Data import DataLoader
dataloader = DataLoader()
dataloader.load_data_from_file(alpha_name="10d_ret_StandardScale_MinMax_10",end_date="2022-01-01",data_path="/home/wuwenjun/feature_platform/ti0/wuwenjun/")
dataloader.feature_data

  1%|          | 15/1705 [00:00<00:11, 145.17it/s]

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


100%|██████████| 1705/1705 [00:11<00:00, 145.01it/s]


10d_ret_StandardScale_MinMax_10
timestamp           ticker                                 
2015-01-05 09:30:00 000001                        -8.111459
                    000002                        -8.453897
                    000004                        -6.767261
                    000005                        -6.968409
                    000006                        -8.406478
...                                                     ...
2021-06-01 09:30:00 688699                        -6.236975
                    688777                        -6.432447
                    688788                        -6.486812
                    688819                        -6.124710
                    688981                        -6.088935

[5281482 rows x 1 columns]

In [8]:
final_df = pd.concat([final_df,dataloader.feature_data],axis=1)
final_df.dropna(axis=0,inplace=True)
final_df.rename(columns={"10d_ret_StandardScale_MinMax_10":"target"},inplace=True)
final_df

0         1            2         3  \
timestamp           ticker                                              
2015-02-16 09:30:00 000001  0.000064 -0.038558  3954.352065 -0.014480   
                    000002  0.000147  0.135621  5523.931352 -0.033847   
                    000004  0.000093 -0.003361   -18.869629 -0.019579   
                    000005  0.000000  0.000000     0.000000  0.000000   
                    000006  0.000035  0.054798  2741.562408 -0.009933   
...                              ...       ...          ...       ...   
2021-05-31 09:30:00 688699  0.000172  0.099482   267.962447 -0.214240   
                    688777  0.000066  0.002534    17.419060 -0.011147   
                    688788  0.000099  0.013165    11.986587 -0.024827   
                    688819  0.000054  0.000613    36.207919 -0.007614   
                    688981  0.000072 -0.005300   196.351503 -0.010879   

                                   4             5         6             7  \
timestamp           ticker                                                   
2015-02-16 09:30:00 000001 -0.153531  -2626.705689 -0.017612  1.415203e+07   
                    000002 -0.583065 -11505.304621 -0.048414  8.801691e+07   
                    000004 -0.011733   -121.588925 -0.039625  6.284071e+03   
                    000005  0.000000      0.000000  0.000000  0.000000e+00   
                    000006  0.008375    430.094542 -0.002161  1.746874e+06   
...                              ...           ...       ...           ...   
2021-05-31 09:30:00 688699 -0.377309   -319.213607 -0.192491  2.288692e+05   
                    688777 -0.006758    -13.315340 -0.018658  2.464624e+03   
                    688788 -0.019989    -24.245655 -0.064414  1.079290e+04   
                    688819 -0.002514    -20.406787 -0.013240  3.404452e+03   
                    688981 -0.005780     54.026622 -0.010531  1.873526e+03   

                                     8             9  ...        111  \
timestamp           ticker                            ...              
2015-02-16 09:30:00 000001   13.814661  1.150742e+05  ...   0.345175   
                    000002  121.464693  2.913541e+06  ...   0.085346   
                    000004    1.512995  1.306707e+04  ...   0.081493   
                    000005    0.000000  0.000000e+00  ...  -1.000000   
                    000006    0.897457 -3.758064e+04  ...   0.088571   
...                                ...           ...  ...        ...   
2021-05-31 09:30:00 688699   74.877938  6.613673e+03  ...  10.997824   
                    688777    0.849211  2.223586e+03  ...  -0.784417   
                    688788    6.958265  5.613754e+03  ...   0.113366   
                    688819    0.648619  3.339437e+03  ...  -0.393657   
                    688981    0.603903 -2.040074e+04  ...   4.595554   

                                 112       113       114       115       116  \
timestamp           ticker                                                     
2015-02-16 09:30:00 000001  0.023481  0.031142 -1.301542  0.014472 -0.012360   
                    000002 -0.009313 -0.004410 -2.271306  0.020087 -0.022228   
                    000004  0.038451 -0.009981 -1.736879  0.011924 -0.014008   
                    000005  0.000000 -1.000000 -1.000000  0.000000  0.000000   
                    000006  0.000000  0.060849 -2.261014  0.013070 -0.012106   
...                              ...       ...       ...       ...       ...   
2021-05-31 09:30:00 688699  0.401769 -0.283446  1.593041  0.029238 -0.053175   
                    688777  0.134618  1.760980 -1.538211  0.031180 -0.027353   
                    688788  0.312307  0.111592 -0.554411  0.017077 -0.041193   
                    688819  0.030769  0.245103 -0.447651  0.013186 -0.011363   
                    688981  0.012709  0.215680  0.372699  0.011055 -0.011121   

                                    117           118       119    target  
timestamp       

In [9]:
from AlphaNet.Data import convert_to_standard_daily_data_par
convert_to_standard_daily_data_par(df = final_df, output_name = "5_Input_20220110_10d_ret_StandardScale_MinMax_10", 
                                   output_path= "/home/wuwenjun/Data_lib/ti0/wuwenjun")

# Pooling ( Save to Pooling)

In [18]:
os.mkdir(path+"Pooling")

In [19]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 229100.64it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [21]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))
    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.columns = [str(i) for i in range(array.shape[1])]
    array.reset_index(inplace=True)
#     # Daily Norm
#     final = []
#     for day in tqdm(array['timestamp'].drop_duplicates()):
#         df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)

#     final = pd.concat(final,axis=0)
#     final.reset_index(inplace=True)
    array.to_parquet(path + "Pooling/" + file)
    print("Save to : ", path + "Pooling/" + file)
    return None
for file in path_list:
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-06-01_2017-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2018-01-01_2018-03-01.par
Save to : 

# Normalize Target and Concat

In [45]:
if os.path.exists(path+"Final") == False:
    os.makedirs(path+"Final")

In [46]:
dataframe_list = pd.read_csv(path + "data.csv").set_index(["timestamp","ticker"])
df = dataframe_list[target].unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=["target"])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(["timestamp","ticker"],inplace=True)
df

target
timestamp           ticker          
2015-01-06 09:30:00 1      -1.126526
                    2      -1.411284
                    4       0.122665
                    5       0.042938
                    6      -0.898595
...                              ...
2021-07-01 09:30:00 688700  2.186672
                    688777  0.631149
                    688788 -1.974877
                    688819  0.325758
                    688981 -0.745459

[5357704 rows x 1 columns]

In [47]:
def test(i):
    dataframe_list = pd.read_parquet(path +"Feat_Concat/" + i).set_index(["timestamp","ticker"])
    dataframe_list = pd.merge(dataframe_list,df,how="left",left_index=True,right_index=True)
    dataframe_list.reset_index().to_parquet(path +"Final/" + i)
    print("Save to : ",path +"Final/" + i)
    return None

In [48]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 27256.16it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2018-01-01_2018-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-09-01_2017-01-01.par
Save to :  Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-03-01_2016-06-01.par/home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2017-06-01_2017-09-01.par

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data

In [49]:
pd.read_parquet(path + "Final/" + "2019-06-01_2019-09-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
0,2019-06-03 09:30:00,1,642.668930,732.445878,90.182527,506.161342,842.470287,127.050138,487.486112,560.657678,...,-0.511487,-0.417024,-0.306245,0.675007,0.596138,0.417038,1.522942,1.344993,0.940935,0.843771
1,2019-06-04 09:30:00,1,2096.386494,548.990224,109.882809,2174.762801,702.839116,171.892827,2208.559264,477.148118,...,-1.709293,-0.320815,-0.686616,0.779296,0.557267,0.501880,1.758233,1.257305,1.132351,0.878615
2,2019-06-05 09:30:00,1,3675.248588,387.817611,329.337998,3527.494588,478.741251,408.572213,3488.742740,237.657136,...,-1.407602,0.007567,-0.692376,0.762180,0.526562,0.522049,1.719625,1.188007,1.177871,0.599032
3,2019-06-06 09:30:00,1,5095.576744,338.420328,330.896680,4833.440429,433.775567,347.356847,4724.327142,211.071372,...,-1.722898,-0.175167,-0.403533,0.728784,0.475635,0.515751,1.644287,1.073122,1.163647,0.392347
4,2019-06-10 09:30:00,1,6144.335576,450.831178,404.073763,6296.420919,502.373333,381.229130,6353.789056,299.574733,...,-2.083133,-0.313553,-0.416336,0.759853,0.446695,0.473500,1.714376,1.007829,1.068298,0.334024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231011,2019-08-26 09:30:00,603999,0.086600,0.485252,4.737556,0.063339,0.418676,4.186172,0.072861,0.462228,...,1.540904,-1.161398,3.116449,0.682538,0.835436,4.423025,1.618849,1.981433,10.490544,-1.556930
231012,2019-08-27 09:30:00,603999,0.227422,0.383404,4.298236,0.137667,0.373142,3.818259,0.184933,0.368693,...,1.497273,-0.786195,1.630575,1.075733,0.732858,4.645142,2.551415,1.738171,11.017364,-1.121929
231013,2019-08-28 09:30:00,603999,0.263048,0.336760,3.575583,0.180887,0.306350,3.306194,0.227586,0.303971,...,1.168284,-0.831340,1.129598,1.208607,0.640404,4.631107,2.866544,1.518891,10.984069,-0.865651
231014,2019-08-29 09:30:00,603999,0.291648,0.236471,2.697064,0.214412,0.201980,2.628958,0.249952,0.211949,...,0.589565,-0.419440,-0.111805,1.204451,0.573242,4.510796,2.856662,1.359589,10.698702,-0.440248
